In [ ]:
#Cài thư viện selenium
!pip install html5lib
!pip install lxml

!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from random import randrange
import json
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import re
import requests

#setting
headers = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)' +
            'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--disable-infobars")
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
options.add_argument('--disable-dev-shm-usage')
options.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 1 
})

Setup cào dữ liệu mục điện thoại

In [ ]:
browser = webdriver.Chrome('chromedriver', options = options)
browser.get('https://www.thegioididong.com/dtdd')
button_show_more = browser.find_element(By.XPATH, '/html/body/section/div[3]/div[2]/a')
while True:
  try:
    button_show_more.click()
    sleep(3)
  except:
    break

html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
browser.close()

Setup cào dữ liệu mục laptop

In [ ]:
browser = webdriver.Chrome('chromedriver', options = options)
browser.get('https://www.thegioididong.com/laptop')
button_show_more = browser.find_element_by_xpath('/html/body/section/div[3]/div[2]/a')
while True:
  try:
    button_show_more.click()
    sleep(3)
  except:
    break
    
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
browser.close()

Function cào toàn bộ đánh giá

In [ ]:
def get_all_reviews(link):
  list_reviews = []
  browser = webdriver.Chrome('chromedriver', options = options)
  browser.get(link)
  sleep(2)
  next_page = 2 
  while True:
    page = browser.page_source
    html = BeautifulSoup(page, 'html.parser')
    reviews = html.findAll('div', class_ = 'comment__item par')
    next_title = 'trang ' + str(next_page)
    for review in reviews:
        name = review.find('p', class_='txtname').text
        content = review.find('p', class_='cmt-txt').text
        star = len(review.findAll('i', class_='icon-star'))
        list_reviews.append({
            'name': name,
            'content': content,
            'star': star
        })
    try:
      button = browser.find_element_by_xpath("//a[@title='"+next_title+"']")
      button.click()
      # print("click page",str(next_page),'sucess')
      next_page += 1
      sleep(3)
    except:
      break
  browser.close()
  return list_reviews
  

Function truy cập vào đường link đánh giá của sản phẩm

In [ ]:
def get_reviews(link):
  #Truy cập vào trang đánh giá
  number_page = 1
  crawl_link = 'https://www.thegioididong.com' + link +'/danh-gia?p='
  get_request = requests.get(crawl_link + str(number_page), headers = headers)
  get_html = BeautifulSoup(get_request.content, 'html.parser')
  #Lấy tên sản phẩm và in ra tên sản phẩm
  product_name = get_html.find('div', class_='box-pdt__content').find('h3').text
  print('Begin collect data from', product_name)
  #Lấy và in ra tiêu đề của đường dẫn hiện tại
  print('title:', get_html.title.text)
  #Khởi tạo mảng chứa các comment sẽ cào được
  list_reviews = []
  while True:
    #Truy cập vào trang đánh giá dựa theo số trang
    get_request = requests.get(crawl_link + str(number_page), headers = headers)
    get_html = BeautifulSoup(get_request.content, 'html.parser')
    #Liệt kê toàn bộ các thẻ div chứa các comment
    reviews = get_html.findAll('div', class_ = 'comment__item par')
    number_page += 1
    #Dừng lại nếu không tìm thấy comment nào
    if len(reviews) == 0:
      break
    #Duyệt qua từng thẻ div chứa comment để lấy thông tin
    for review in reviews:
      #Lấy tên người dùng
      name = review.find('p', class_='txtname').text
      #Lấy nội dung comment
      content = review.find('p', class_='cmt-txt').text
      #Lấy đánh giá sao
      star = len(review.findAll('i', class_='icon-star'))
      #Đưa vào mảng chứa comment
      list_reviews.append({
          'name': name,
          'content': content,
          'star': star,
      })
    sleep(3)
  print('Collect data from', product_name, 'is done!, found', str(len(list_reviews)), 'comments')
  print('----------------------------------------------------')
  return list_reviews

Function lấy toàn bộ các đánh giá về sản phẩm

Thu thập đánh giá từ mục điện thoại



In [ ]:
list_products = soup.findAll('a', {'class':'main-contain'})
list_reviews = []
list_links = [link.attrs['href'] for link in list_products]
for link in list_links:
  list_reviews += get_reviews(link[:len(link) - 8])

#Lưu lại thành file json
with open('data_TGDD.json', 'w', encoding='utf-8') as file:
 json.dump(list_reviews, file)

246


Thu thập đánh giá từ mục laptop

In [ ]:
list_products = soup.findAll('a', {'class':'main-contain'})
g_list_laptops = []
list_links = [link.attrs['href'] for link in list_products]
link = list_links[1]
error = []
for link in list_links:
  try:
    g_list_laptops.append(get_review(link[:len(link) - 8]))
  except:
    error.append(link)

with open('laptop_data.json', 'w', encoding='utf-8') as file:
 json.dump(g_list_laptops, file)